In [1]:
import pandas as pd
from os.path import join, isfile
from os import listdir
import numpy as np

In [3]:
datafiles = [join('.\\data', f) for f in listdir('./data') if join('./data', f).endswith(".csv")]

In [83]:
counter = 0
learning_frames = []
recall_frames = []
IDs = []
triggers = []
for f in datafiles:
    
    if "end_part2_text.started" in pd.read_csv(f):
        df = pd.read_csv(f, converters={'ID-Nummer': str})
        IDs.append(df["ID-Nummer"][0])
        end_start = df["wait_for_trigger.stopped"][df["starting_soon.started"].notna()].values
        triggers = [end_start[0], end_start[1], end_start[3], end_start[4]]

        # LEARNING
        learning = df.loc[df["learning_trials.thisIndex"] >= 0]
        learning_frames.append(learning)

        
        # RECALL
        recall = df.loc[df["test_trials.thisIndex"] >= 0]
        recall_frames.append(recall)
        counter += 1
print('Number of finished experiments', counter)

Number of finished experiments 1


In [113]:
triggers

[147.51405999999952, 782.0453943000002, 1463.4908693999996, 2097.621641]

In [172]:
for frame in learning_frames:
    guess_times1 = frame.loc[frame["trial_type"]=="Guess","guess.started"] - triggers[0]
    guess_times2 = frame.loc[frame["trial_type"]=="Guess","guess.started"] - triggers[1]
    guess_times1 = guess_times1.values[guess_times2.values < 0]
    guess_times2 = guess_times2.values[guess_times2.values >= 0]
    print(guess_times1)
    print(guess_times2)
    data = np.array([guess_times1, np.ones(len(guess_times1)), np.ones(len(guess_times1)) * 3]).T
    guess1_frame = pd.DataFrame(data=data, columns = ['Onsets', 'Dur', 'Val'])

    guess_reached_times1 = frame.loc[frame["trial_type"]=="Guess","guess_reached.started"] - triggers[0]
    guess_reached_times2 = frame.loc[frame["trial_type"]=="Guess","guess_reached.started"] - triggers[1]
    guess_reached_times1 = guess_reached_times1.values[guess_reached_times2.values < 0]
    guess_reached_times2 = guess_reached_times2.values[guess_reached_times2.values >= 0]
    print(guess_reached_times1)
    print(guess_reached_times2)

    encode_guess_times1 = frame.loc[frame["trial_type"]=="Guess","encode.started"] - triggers[0]
    encode_guess_times2 = frame.loc[frame["trial_type"]=="Guess","encode.started"] - triggers[1]
    encode_guess_times1 = encode_guess_times1.values[encode_guess_times2.values < 0]
    encode_guess_times2 = encode_guess_times2.values[encode_guess_times2.values >= 0]
    print(encode_guess_times1)
    print(encode_guess_times2)

[  9.0278925  24.0207388  55.5377503  70.5496497  85.5502401 103.5623237
 118.56473   151.5977238 166.5949145 184.5943843 202.608049  219.1093575
 235.6084557 273.1384066 288.1389748 304.6347567 346.6532562 363.1520538
 405.215906  420.2286836 439.730644  501.3104791]
[4.20530000e-03 1.80054275e+01 8.40638822e+01 1.02083607e+02
 1.30597948e+02 1.66643175e+02 2.04173906e+02 2.32691958e+02
 2.82239455e+02 2.98739823e+02 3.15252792e+02 3.43769718e+02
 3.82799968e+02 3.97798550e+02 4.21831347e+02 4.41331158e+02
 4.78878183e+02 5.04376921e+02]
[ 12.0325768  27.031242   58.5462229  73.5614217  88.5605529 106.5767296
 121.5754229 154.6073264 169.6063289 187.6048638 205.6207026 222.1203612
 238.619611  276.1509963 291.1493385 307.6485518 349.663008  366.162855
 408.2269674 423.2423216 442.740918  504.3213435]
[  3.0159374  21.0153324  87.0775719 105.0936645 133.608525  169.6567604
 207.1880379 235.7032672 285.2499927 301.7496273 318.2646417 346.7799728
 385.8114083 400.8100304 424.8424563 444.